# Search Algorithms

In this project, the aim is to solve a predefined problem using various search algorithms and compare their qualities. BFS, IDS and A* are the algorithms used for this problem.

## Problem Description

TBA

## Modeling

At first, the problem should be modeled as a graph so that the search algorithms can be applied to it in future steps. Each node representes one ditinct state of the map. Bellow, the components of the model is explained:

 - <b>Initial State:</b> The initial state of the map is given as input in this problem, and the player always starts at bottom left corner of the map.
 
 
 - <b>Actions:</b> At each step,one of the players can move either up, down, left or right. However, sometimes the players may be prevented from taking certain actions due to walls and obstacles.
 
 
 - <b>Transition Model:</b> TBA
 
 
 - <b>Goal State:</b> The goal state is when all of the players have reached the top right corner of the map and collected all potions in the map.
 
 
 - <b>Path Cost:</b> TBA
 

### Model Implementation

In [1]:
PLAYER='P'
POTION='M'
PILL='D'
OBSTACLE='O'

 - <b>Initiat State:</b>

In [2]:
def read_initial_state(file_name):
    file = open(file_name)
    row,col=file.readline().split()
    potion_count,pill_count=file.readline().split()
    initial_map={"row": int(row) , "col": int(col) , "potion_count": int(potion_count), "ploc":[]}
    for _ in range(int(potion_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=POTION
    for _ in range(int(pill_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=PILL
    obstacle_count=file.readline()
    for _ in range(int(obstacle_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=OBSTACLE
    initial_map["ploc"].append((0,0))
    return initial_map

 - <b>Actions:</b>

In [3]:
UP=(1,0)
DOWN=(-1,0)
RIGHT=(0,1)
LEFT=(0,-1)
ACTIONS=[UP,DOWN,LEFT,RIGHT]
ACT_DICT={UP:"U",DOWN:"D",RIGHT:"R",LEFT:"L"}

 - <b>Transition Model:</b>

In [4]:
import copy as cp
def not_in_range(loc,loc_range):
    return loc[0]>=loc_range[0] or loc[1]>=loc_range[1] or loc[0]<0 or loc[1]<0
    

def move(current_state,player_number,direction):
    current_state=cp.deepcopy(current_state)
    current_location=current_state["ploc"][player_number]
    future_location=(current_location[0]+direction[0],current_location[1]+direction[1])
    if(future_location in current_state):
        if(current_state[future_location]==POTION):
            current_state["potion_count"]-=1;
        elif(current_state[future_location]==PILL):
            current_state["ploc"].append((current_state["row"]-1,0))
        elif(current_state[future_location]==OBSTACLE):
            return 0
    elif(not_in_range(future_location,(current_state["row"],current_state["col"]))):
        return 0
    current_location=current_state["ploc"][player_number]=future_location
    if(current_location in current_state):
        del current_state[current_location]
    return current_state

In [5]:
def is_goal_state(state):
    for loc in state["ploc"]:
        if(loc!=(state["row"]-1,state["col"]-1)):
            return False
    if(state["potion_count"]==0):
        return True
    else:
        return False

In [6]:
import copy as cp
c=read_initial_state("Tests/test1.in")
d=cp.deepcopy(c)
d=move(cp.deepcopy(d),0,(0,1))
d=move(cp.deepcopy(d),0,(0,1))
d=move(cp.deepcopy(d),0,(0,1))
d=move(cp.deepcopy(d),0,(1,0))
print(d)

{'row': 4, 'col': 4, 'potion_count': 1, 'ploc': [(1, 3), (3, 0)], (2, 0): 'M', (1, 2): 'O'}


# BFS

Explainations TBA

In [142]:
def encode(state):
    encoded={}
    for player in range(len(state["ploc"])):
        encoded[player]=state["ploc"][player]
    encoded["potion_count"]=state["potion_count"]
    return tuple(encoded.items())

def BFS(file_address):
    i=0;
    frontier_queue=[]
    visited_states={}
    frontier_queue.append(read_initial_state(file_address))
    visited_states[encode(frontier_queue[0])]=True
    frontier_queue[0]["order"]=""
    while(frontier_queue):
        current_state=frontier_queue.pop(0)
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                if((encode(new_state) in visited_states)):
                    continue
                visited_states[encode(new_state)]=True
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                if(is_goal_state(new_state)):
                    return new_state["order"]
                frontier_queue.append(new_state) 
    return 0

In [143]:
print(BFS("Tests/test3.in"))

 0U 0U 0R 0R 0D 0R 0R 0R 0U 0U 0U 0U 1R 1R 1R 1D 1U 1R 1R


# IDS

TBA

In [140]:
def DFS(initial_state,max_level):
    i=0;
    frontier_queue=[]
    visited_states={}
    frontier_queue.append([0,initial_state])
    #visited_states[encode(frontier_queue[0])]=1
    initial_state["order"]=""
    while(frontier_queue):
        added_level,current_state=frontier_queue.pop()
        if((encode(current_state) in visited_states)):
            if(visited_states[encode(current_state)]<=added_level):
                continue
        visited_states[encode(current_state)]=added_level
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                if(is_goal_state(new_state)):
                    return new_state["order"]
                if(added_level+1<max_level):
                    frontier_queue.append([added_level+1,new_state]) 
    return 0
def IDS(file_address):
    level=1
    start_state=read_initial_state(file_address)
    start_state["order"]=""
    res=DFS(start_state,level)
    while res==0:
        #print(level)
        level+=1
        res=DFS(start_state,level)
    return res

In [141]:
print(IDS("Tests/test1.in"))

 0R 1D 1R 1R 1R 0R 1U 0R 0U 0U 0U


# A*

TBA

In [ ]:
heuristic(state){
    player_steps=0
    min_row=state["row"]-1
    min_col=state["col"]-1
    col=min_col
    row=min_row
    for loc in state["ploc"]:
        player_steps=row-loc[0]+col-loc[1]
    for key,val in state.items():
        if (type(key)==tuple):
            
}